In [1]:
import glob
from google.colab import drive
drive.mount('/content/drive')

base_path = "/content/drive/My Drive/Poemes"

for path in glob.glob(base_path):
  print(path)

Mounted at /content/drive
/content/drive/My Drive/Poemes


In [2]:
!pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [4]:
#On a besoin du packet sacraemoses pour le tokenizer du flaubert https://pypi.org/project/sacremoses/

!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 11.1 MB/s eta 0:00:00


In [5]:
# Avec le Flaubert 'flaubert/flaubert_large_cased" de Hugging Face

from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import os
import json
from tqdm import tqdm
model_name = "flaubert/flaubert_large_cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

nlp_ner = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.56M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/896k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.49G [00:00<?, ?B/s]

Some weights of FlaubertForTokenClassification were not initialized from the model checkpoint at flaubert/flaubert_large_cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


In [6]:
import os
import json
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

from google.colab import drive
drive.mount('/content/drive')

model_checkpoint = "flaubert/flaubert_base_cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Como FlauBERT não foi treinado diretamente para NER, então vamos carregar uma cabeça de token classification básica
# Como workaround: carregamos o modelo como se fosse para Token Classification (não é o ideal para produção sem fine-tuning), é como enganar o hugging face
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=2
)

nlp_ner = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

base_path = "/content/drive/My Drive/Poemes/"
dic = {}

entites_par_auteur = {}
tokens_par_auteur = {}

for autor in tqdm(os.listdir(base_path), desc="Autors", unit="auteur"):
    autor_path = os.path.join(base_path, autor)

    if os.path.isdir(autor_path):
        entites_par_auteur[autor] = 0
        tokens_par_auteur[autor] = 0

        for poema in tqdm(os.listdir(autor_path), desc=f"Poèmes de {autor}", unit="poème", leave=False):
            if poema.endswith(".txt"):
                file_path = os.path.join(autor_path, poema)

                with open(file_path, "r", encoding="utf-8") as f:
                    conteudo = f.read()

                tokens = tokenizer.tokenize(conteudo)
                nb_tokens = len(tokens)

                entidades = nlp_ner(conteudo)
                nb_entites = len(entidades)

                tokens_par_auteur[autor] += nb_tokens
                entites_par_auteur[autor] += nb_entites

                dic[poema] = {
                    "auteur": autor,
                    "nombre de tokens": nb_tokens,
                    "nombre d'entités nommées": nb_entites,
                    "entités nommées": [{"texte": ent["word"], "type": ent["entity_group"]} for ent in entidades],
                }

json_file_path = "/content/drive/My Drive/resultat_flaubert_seul.json"
with open(json_file_path, "w", encoding="utf-8") as json_file:
    json.dump(dic, json_file, ensure_ascii=False, indent=4)

print("\nNombre total de tokens par auteur :")
for auteur, nb_tokens in tokens_par_auteur.items():
    print(f"- {auteur}: {nb_tokens} tokens")

print("\nNombre total d'entités nommées par auteur :")
for auteur, nb_entites in entites_par_auteur.items():
    print(f"- {auteur}: {nb_entites} entités nommées")

total_tokens = sum(tokens_par_auteur.values())
total_entites = sum(entites_par_auteur.values())

print(f"\nNombre total de tokens dans tous les poèmes : {total_tokens}")
print(f"Nombre total d'entités nommées dans tous les poèmes : {total_entites}")

print(f"\nAnalyse terminée. Résultats enregistrés dans {json_file_path}.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


tokenizer_config.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.56M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/896k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/553M [00:00<?, ?B/s]

Some weights of FlaubertForTokenClassification were not initialized from the model checkpoint at flaubert/flaubert_base_cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu
Autors: 100%|██████████| 10/10 [00:32<00:00,  3.25s/auteur]


Nombre total de tokens par auteur :
- Appolinaire: 19808 tokens
- Darbourville: 60937 tokens
- Desbordes: 35102 tokens
- Hugo: 75902 tokens
- Loiseau: 22432 tokens
- Noialles: 14641 tokens
- Rimbaud: 23008 tokens
- Sauvage: 24103 tokens
- Verlaine: 15719 tokens
- Vivien: 45851 tokens

Nombre total d'entités nommées par auteur :
- Appolinaire: 198 entités nommées
- Darbourville: 192 entités nommées
- Desbordes: 185 entités nommées
- Hugo: 179 entités nommées
- Loiseau: 196 entités nommées
- Noialles: 205 entités nommées
- Rimbaud: 206 entités nommées
- Sauvage: 192 entités nommées
- Verlaine: 180 entités nommées
- Vivien: 212 entités nommées

Nombre total de tokens dans tous les poèmes : 337503
Nombre total d'entités nommées dans tous les poèmes : 1945

Analyse terminée. Résultats enregistrés dans /content/drive/My Drive/resultat_flaubert_seul.json.


In [8]:
import os
import json
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

from google.colab import drive
drive.mount('/content/drive')

model_checkpoint = "flaubert/flaubert_base_cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Como FlauBERT não foi treinado diretamente para NER, então vamos carregar uma cabeça de token classification básica
# Como workaround: carregamos o modelo como se fosse para Token Classification (não é o ideal para produção sem fine-tuning), é como enganar o hugging face
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=2
)

nlp_ner = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

base_path = "/content/drive/My Drive/Poemes/"
dic = {}

entites_par_auteur = {}
tokens_par_auteur = {}

for autor in tqdm(os.listdir(base_path), desc="Autors", unit="auteur"):
    autor_path = os.path.join(base_path, autor)

    if os.path.isdir(autor_path):
        entites_par_auteur[autor] = 0
        tokens_par_auteur[autor] = 0

        for poema in tqdm(os.listdir(autor_path), desc=f"Poèmes de {autor}", unit="poème", leave=False):
            if poema.endswith(".txt"):
                file_path = os.path.join(autor_path, poema)

                with open(file_path, "r", encoding="utf-8") as f:
                    conteudo = f.read()

                tokens = tokenizer.tokenize(conteudo)
                nb_tokens = len(tokens)

                entidades = nlp_ner(conteudo)
                nb_entites = len(entidades)

                tokens_par_auteur[autor] += nb_tokens
                entites_par_auteur[autor] += nb_entites

                dic[poema] = {
                    "auteur": autor,
                    "nombre de tokens": nb_tokens,
                    "nombre d'entités nommées": nb_entites,
                    "entités nommées": [{"texte": ent["word"], "type": ent["entity_group"]} for ent in entidades],
                }

json_file_path = "/content/drive/My Drive/resultat_flaubert_seul.json_02"
with open(json_file_path, "w", encoding="utf-8") as json_file:
    json.dump(dic, json_file, ensure_ascii=False, indent=4)

print("\nNombre total de tokens par auteur :")
for auteur, nb_tokens in tokens_par_auteur.items():
    print(f"- {auteur}: {nb_tokens} tokens")

print("\nNombre total d'entités nommées par auteur :")
for auteur, nb_entites in entites_par_auteur.items():
    print(f"- {auteur}: {nb_entites} entités nommées")

total_tokens = sum(tokens_par_auteur.values())
total_entites = sum(entites_par_auteur.values())

print(f"\nNombre total de tokens dans tous les poèmes : {total_tokens}")
print(f"Nombre total d'entités nommées dans tous les poèmes : {total_entites}")

print(f"\nAnalyse terminée. Résultats enregistrés dans {json_file_path}.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of FlaubertForTokenClassification were not initialized from the model checkpoint at flaubert/flaubert_base_cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu
Autors: 100%|██████████| 10/10 [00:30<00:00,  3.02s/auteur]


Nombre total de tokens par auteur :
- Appolinaire: 19808 tokens
- Darbourville: 60937 tokens
- Desbordes: 35102 tokens
- Hugo: 75902 tokens
- Loiseau: 22432 tokens
- Noialles: 14641 tokens
- Rimbaud: 23008 tokens
- Sauvage: 24103 tokens
- Verlaine: 15719 tokens
- Vivien: 45851 tokens

Nombre total d'entités nommées par auteur :
- Appolinaire: 225 entités nommées
- Darbourville: 165 entités nommées
- Desbordes: 181 entités nommées
- Hugo: 199 entités nommées
- Loiseau: 186 entités nommées
- Noialles: 183 entités nommées
- Rimbaud: 200 entités nommées
- Sauvage: 213 entités nommées
- Verlaine: 177 entités nommées
- Vivien: 205 entités nommées

Nombre total de tokens dans tous les poèmes : 337503
Nombre total d'entités nommées dans tous les poèmes : 1934

Analyse terminée. Résultats enregistrés dans /content/drive/My Drive/resultat_flaubert_seul.json_02.


In [9]:
import os
import json
from tqdm import tqdm
from transformers import AutoTokenizer, pipeline

tokenizer = AutoTokenizer.from_pretrained("flaubert/flaubert_base_cased")
nlp_ner = pipeline("ner", model="Jean-Baptiste/camembert-ner")

base_path = "/content/drive/My Drive/Poemes/"
dic = {}

entites_par_auteur = {}
tokens_par_auteur = {}

for autor in tqdm(os.listdir(base_path), desc="Autors", unit="auteur"):
    autor_path = os.path.join(base_path, autor)

    if os.path.isdir(autor_path):
        entites_par_auteur[autor] = 0
        tokens_par_auteur[autor] = 0

        total_poemes = len([poema for poema in os.listdir(autor_path) if poema.endswith(".txt")])
        poems_processed = 0

        for poema in tqdm(os.listdir(autor_path), desc=f"Poèmes de {autor}", unit="poème", leave=False):
            if poema.endswith(".txt"):
                file_path = os.path.join(autor_path, poema)

                with open(file_path, "r", encoding="utf-8") as f:
                    conteudo = f.read()

                tokens = tokenizer.tokenize(conteudo)
                nb_tokens = len(tokens)

                try:

                    entidades = nlp_ner(conteudo)
                    nb_entites = len(entidades)

                    tokens_par_auteur[autor] += nb_tokens
                    entites_par_auteur[autor] += nb_entites

                    dic[poema] = {
                        "auteur": autor,
                        "nombre de tokens": nb_tokens,
                        "nombre d'entités nommées": nb_entites,
                        "entités nommées": [{"texte": ent["word"], "type": ent["entity_group"]} for ent in entidades],
                    }

                    poems_processed += 1

                except Exception as e:
                    print(f"❗ Erreur lors du traitement de {poema} ({autor}) : {e}")

        if poems_processed != total_poemes:
            print(f"⚠️ Attention : certains poèmes de {autor} n'ont pas été traités correctement.")

json_file_path = "/content/drive/My Drive/resultat_flaubert_seul_03.json"
with open(json_file_path, "w", encoding="utf-8") as json_file:
    json.dump(dic, json_file, ensure_ascii=False, indent=4)

print("\nNombre total de tokens par auteur :")
for auteur, nb_tokens in tokens_par_auteur.items():
    print(f"- {auteur}: {nb_tokens} tokens")

print("\nNombre total d'entités nommées par auteur :")
for auteur, nb_entites in entites_par_auteur.items():
    print(f"- {auteur}: {nb_entites} entités nommées")

total_tokens = sum(tokens_par_auteur.values())
total_entites = sum(entites_par_auteur.values())

print(f"\nNombre total de tokens dans tous les poèmes : {total_tokens}")
print(f"Nombre total d'entités nommées dans tous les poèmes : {total_entites}")

print(f"\n✅ Analyse terminée. Résultats enregistrés dans {json_file_path}.")


config.json:   0%|          | 0.00/892 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/269 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

...

❗ Erreur lors du traitement de SAUVAGE_Tandis-que-la-terre-tourne.txt (Sauvage) : 'entity_group'
⚠️ Attention : certains poèmes de Sauvage n'ont pas été traités correctement.



Autors:  90%|█████████ | 9/10 [00:23<00:02,  2.33s/auteur]

❗ Erreur lors du traitement de VERLAINE_Sagesse.txt (Verlaine) : 'entity_group'
⚠️ Attention : certains poèmes de Verlaine n'ont pas été traités correctement.



Autors: 100%|██████████| 10/10 [00:25<00:00,  2.56s/auteur]

❗ Erreur lors du traitement de VIVIEN_Etudes-et-preludes.txt (Vivien) : 'entity_group'
⚠️ Attention : certains poèmes de Vivien n'ont pas été traités correctement.

Nombre total de tokens par auteur :
- Appolinaire: 19808 tokens
- Darbourville: 60937 tokens
- Desbordes: 35102 tokens
- Hugo: 75902 tokens
- Loiseau: 22432 tokens
- Noialles: 14641 tokens
- Rimbaud: 23008 tokens
- Sauvage: 24103 tokens
- Verlaine: 15719 tokens
- Vivien: 45851 tokens

Nombre total d'entités nommées par auteur :
- Appolinaire: 75 entités nommées
- Darbourville: 127 entités nommées
- Desbordes: 54 entités nommées
- Hugo: 94 entités nommées
- Loiseau: 87 entités nommées
- Noialles: 101 entités nommées
- Rimbaud: 91 entités nommées
- Sauvage: 90 entités nommées
- Verlaine: 125 entités nommées
- Vivien: 157 entités nommées

Nombre total de tokens dans tous les poèmes : 337503
Nombre total d'entités nommées dans tous les poèmes : 1001

✅ Analyse terminée. Résultats enregistrés dans /content/drive/My Drive/resulta